# Creating Kaggle submissions for CIFAR10 rotation classification (Parts 1/4a)

In [1]:
# you will be prompted with a window asking to grant permissions
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
import os
datadir = "/content/assignment3"
if not os.path.exists(datadir):
  !ln -s "/content/drive/My Drive/assignment3_starter_sp25" $datadir # TODO: Fill your assignment3 path
os.chdir(datadir)
!pwd

/content/drive/My Drive/assignment3_starter_sp25


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np


def rotate_img(img, rot):
    """
    Implement the same rotation logic as in your code
    """
    if rot == 0: # 0 degrees rotation
        return img
    # TODO: Implement rotate_img() - return the rotated img
    #
    #
    #
    # else:
    #     raise ValueError('rotation should be 0, 90, 180, or 270 degrees')
    return torch.rot90(img, k=rot, dims=(1, 2))


class CIFAR10RotationTestOnly(torchvision.datasets.CIFAR10):

    def __init__(self, root, download, transform) -> None:
        super().__init__(root=root, train=False, download=download, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        image, cls_label = super().__getitem__(index)
        image_rotated = rotate_img(image, index % 4)

        return image_rotated

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

batch_size = 128

testset = CIFAR10RotationTestOnly(root='./data', download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
@torch.inference_mode()
def run_rotation_test(net, testloader):
    all_output = []
    for images in testloader:
        images = images.to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        all_output.append(predicted.cpu().numpy())
    return np.concatenate(all_output)

## Instantiate your model here

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class ModifiedResNet18(nn.Module):
    def __init__(self, num_classes=4):
        super(ModifiedResNet18, self).__init__()
        self.pre_conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )


        self.resnet = resnet18()
        self.resnet.conv1 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)


        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.pre_conv(x)
        x = self.resnet(x)
        return x

In [7]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18


net = ModifiedResNet18(num_classes=4)
net = net.to(device)
net.eval()

ModifiedResNet18(
  (pre_conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (resnet): ResNet(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

## Load the model weights

In [8]:
PATH = './ModifiedResNet18.pth'
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

## Submit the generated csv file to Kaggle

In [9]:
import pandas as pd
predictions = run_rotation_test(net, testloader)
ids = np.arange(len(predictions))
df = pd.DataFrame({'id': ids, 'prediction': predictions})
df.to_csv('./rotation_test_predictions.csv', index=False)